## Ideia


Vamos extrair os dados sobre alunos, cursos e IES de ensino superior do INEP de 2009 a 2019. Para isto precisaremos extrair os dados do site do INEP(felizmente já temos as URL's. Depois faremos uma limpeza nos dados duplicados, para deixar a base de dados mais limpa e criar um dataset para análise que será armazena no google drive. Vamos também cronometar o tempo que o script demora para executar.

##Extraindo os dados do INEP

Instalando bibliotecas necessárias.

In [ ]:
!pip install pyunpack
!pip install dask[dataframe]
!pip install patool


     |████████████████████████████████| 102kB 2.7MB/s 
  Created wheel for locket: filename=locket-0.2.0-cp36-none-any.whl size=4042 sha256=09c72a9660cb409b4b32c0e6491384c77b13ae3fc0544a82e17d13f887ce34ec
  Stored in directory: /root/.cache/pip/wheels/26/1e/e8/4fa236ec931b1a0cdd61578e20d4934d7bf188858723b84698
Successfully built locket
     |████████████████████████████████| 81kB 3.8MB/s 


Importando Bibliotecas

In [ ]:
# importando bibliotecas

import pandas as pd
import zipfile
import os
from pyunpack import Archive
import dask.dataframe as dd
import warnings
warnings.filterwarnings("ignore")
import shutil
from datetime import datetime
startTime = datetime.now()

In [ ]:
#@title Criando variável contendo as urls de interesse
urls  = ["http://download.inep.gov.br/microdados/microdados_censo_superior_2009.zip",
         "http://download.inep.gov.br/microdados/microdados_censo_superior_2010.zip",
         "http://download.inep.gov.br/microdados/microdados_censo_superior_2011.zip",
         "http://download.inep.gov.br/microdados/microdados_censo_superior_2012.zip",
         "http://download.inep.gov.br/microdados/microdados_censo_superior_2013.zip",
         "http://download.inep.gov.br/microdados/microdados_censo_superior_2014.zip",
         "http://download.inep.gov.br/microdados/microdados_censo_superior_2015.zip",
         "http://download.inep.gov.br/microdados/microdados_censo_superior_2016.zip",
         "http://download.inep.gov.br/microdados/microdados_educacao_superior_2017.zip",
         "http://download.inep.gov.br/microdados/microdados_educacao_superior_2018.zip",
         "http://download.inep.gov.br/microdados/microdados_educacao_superior_2019.zip"]




In [ ]:
#@title Criando função para coletar os dados
def DownloadData(url) :
   import os 
   os.system(f'wget {url}')


Coletando os dados :

In [ ]:


for i in (range(len(urls))):
  DownloadData(urls[i])



Os arquivos baixados são arquivos zip, precisamos descompactá-los em diretórios para trabalhar. Faremos isto em seguida. Foi escolhida para o diretório o ano dos dados( de 2009 a 2019). Para facilitar foi criado algumas funções para descompactar os arquivos .zip e criar as rotas onde os dados serão armazenados.

Criando os nomes dos diretórios e arquivos zip

In [ ]:
diretorio = ["2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019"]
zip_files = ["microdados_censo_superior_2009.zip","microdados_censo_superior_2010.zip",
             "microdados_censo_superior_2011.zip","microdados_censo_superior_2012.zip",
             "microdados_censo_superior_2013.zip","microdados_censo_superior_2014.zip",
             "microdados_censo_superior_2015.zip","microdados_censo_superior_2016.zip",
             "microdados_educacao_superior_2017.zip","microdados_educacao_superior_2018.zip",
             "microdados_educacao_superior_2019.zip"]



In [ ]:
#@Criando as funções
def criandoenderecos(diretorios,arquivos) :
 for number in range(len(diretorios)) :
  diretorio[number] = f"/content/{diretorio[number]}"
  arquivos[number ] = f"/content/{zip_files[number]}"

def criandodiretorio(diretorio) :
  import os
  for number in range(len(diretorio)) :
    os.mkdir(diretorio[number])

def criandoarquivo(diretorio,arquivo):
  import zipfile 
  with zipfile.ZipFile(arquivo, 'r') as zip_ref:
    zip_ref.extractall(diretorio)

Aplicando as funções

In [ ]:
criandoenderecos(diretorio,zip_files) # ciranco os endereços de acesso
criandodiretorio(diretorio) #criando os diretorios


In [ ]:

for i in (range(len(diretorio))) :
  criandoarquivo(diretorio[i],zip_files[i]) #unpack dos arquivos zip


Os arquivos que nos interessam estão em vários formatos (zip,rar,csv) entretanto seguem um padrão. Usaremos isto para descompactar os arquivos rar e zip e juntá-los numa variável.
Também renomearemos os dados referentes a 2019, pois fogem do padrão esperado.

In [ ]:
dado = "DM_ALUNO"

dado_inicio = "/content/2019/Microdados_Educaç╞o_Superior_2019/dados/SUP_ALUNO_2019.CSV"
dado_final = f"/content/2019/Microdados_Educaç╞o_Superior_2019/dados/{dado}.CSV"
os.rename(dado_inicio,dado_final)
arc_zip =  ! find . | grep -i {dado}.zip
arc_rar =  ! find . | grep -i {dado}.rar



Vamos dar unpack nos dados rar e nos dados zip. Felizmente já temos uma função que realiza o zip. Vamos criar uma para os arquivos .rar

In [ ]:
#@title Criando função pra descompactar arquivos .rar

def descompactaArquivosRar(arquivosrarcomeco) :
  Archive(arquivosrarcomeco).extractall(arquivosrarcomeco[2:6])


Aplicando a função

In [ ]:

for i in (range(len(arc_rar))) :
  descompactaArquivosRar(arc_rar[i])

Descompactando os arquivos zip

In [ ]:

for i in (range(len(arc_zip))) :
  criandoarquivo (arc_zip[i][2:6], arc_zip[i])
  

  

Vamos confirmar se temos todos arquivos que queremos.

In [ ]:
arquivos = ! find . | grep -i {dado}.csv
arquivos= arquivos.sort()
arquivos



['./2009/DM_ALUNO.CSV',
 './2010/DM_ALUNO.CSV',
 './2011/DM_ALUNO.CSV',
 './2012/DM_ALUNO.CSV',
 './2013/DM_ALUNO.CSV',
 './2014/microdados_censo_superior_2014/2014/DADOS/DM_ALUNO.CSV',
 './2015/DM_ALUNO.CSV',
 './2016/DM_ALUNO.CSV',
 './2017/DM_ALUNO.CSV',
 './2018/microdados_ed_superior_2018/dados/DM_ALUNO.CSV',
 './2019/Microdados_Educaç╞o_Superior_2019/dados/DM_ALUNO.CSV']

Temos os arquivos que nos interessam para Aluno. Precisamos então repetir o processo para arquivos do tipo DM_IES e DM_CURSO. Vamos criar uma função para retornar a lista já descompactada.

In [ ]:
def geraTodosArquivos(dados) :

  dado_inicio =f"/content/2019/Microdados_Educaç╞o_Superior_2019/dados/SUP_{dados[3:]}_2019.CSV"
  dado_final = f"/content/2019/Microdados_Educaç╞o_Superior_2019/dados/{dados}.CSV"
  os.rename(dado_inicio,dado_final)

  arc_zip =  ! find . | grep -i {dados}.zip
  arc_rar =  ! find . | grep -i {dados}.rar
  for i in (range(len(arc_zip))) :
    criandoarquivo (arc_zip[i][2:6], arc_zip[i])
    print(arc_zip[i])

  for i in (range(len(arc_rar))) :
    descompactaArquivosRar(arc_rar[i])
  
  arquivos = ! find . | grep -i {dados}.csv
  arquivos= arquivos.sort()
  return arquivos


In [ ]:
arquivos_IES = geraTodosArquivos("DM_IES")


./2017/Microdados_Educacao_Superior_2017/DADOS/DM_IES.zip


In [ ]:
arquivos_IES

['./2009/DM_IES.CSV',
 './2010/DM_IES.CSV',
 './2011/DM_IES.CSV',
 './2012/DM_IES.CSV',
 './2013/DM_IES.CSV',
 './2014/microdados_censo_superior_2014/2014/DADOS/DM_IES.CSV',
 './2015/DM_IES.CSV',
 './2016/DM_IES.CSV',
 './2017/DM_IES.CSV',
 './2018/microdados_ed_superior_2018/dados/DM_IES.CSV',
 './2019/Microdados_Educaç╞o_Superior_2019/dados/DM_IES.CSV']

In [ ]:
arquivos_CURSO = geraTodosArquivos("DM_CURSO")

./2017/Microdados_Educacao_Superior_2017/DADOS/DM_CURSO.zip


In [ ]:
arquivos_CURSO

['./2009/DM_CURSO.CSV',
 './2010/DM_CURSO.CSV',
 './2011/DM_CURSO.CSV',
 './2012/DM_CURSO.CSV',
 './2013/DM_CURSO.CSV',
 './2014/microdados_censo_superior_2014/2014/DADOS/DM_CURSO.CSV',
 './2015/DM_CURSO.CSV',
 './2016/DM_CURSO.CSV',
 './2017/DM_CURSO.CSV',
 './2018/microdados_ed_superior_2018/dados/DM_CURSO.CSV',
 './2019/Microdados_Educaç╞o_Superior_2019/dados/DM_CURSO.CSV']

##Mexendo com Alunos

Agora que temos todos os arquivos que precisamos, vamos juntá-los em apenas um arquivo. Os arquivos com DM_ALUNO são grandes e para isto, vamos utilizar a biblioteca Dask para não estourar a ram do sistema. Vamos pegar os dados referentes ao Aluno que nos interessa, no meu caso serão os dados abaixo :

* CO_IES - Instituição de ensino na qual o Aluno está matriculado.

* CO_ALUNO  - Código de Identificação Único do Aluno

* Cat_Adm - Categoria Administrativa da faculdade de ensino do aluno     ( Publica,Privada e seus derivados)

* CO_CURSO - Curso do Aluno

* Nivel_Aca - Nível Acadêmico do Aluno ( Graduação ou superior)

* Genêro - Gênero do Aluno (Masculino ou Feminino)

* Cor/Raça = Cor/Raça do Aluno

* IN_Resrva_Etnico : Indica se o aluno entrou na faculdade atráves de reserva de vagas de cunho etníco

* IN_Reserva_Vagas : Indica se o aluno entrou na atráves de reserva de vagas

* Apoio_Social = Indica se o aluno recebe algum tipo de Apoio Social.

* Ano - Ano que o dado foi coletado.



Como houve mudanças nas nomeclaturas adotadas pelo INEP, foi necessário criar mais de uma variável para coletar os dados, depedendo do arquivo que ela está lendo.  Por exemplo, em 2009 temos que o código identificador do Aluno é indicado por CO_ALUNO e em 2019 o dado já é contabilizado como ID_ALUNO

In [ ]:
#@title Definindo algumas variáveis utilizadas
types = {'CO_GRAU_ACADEMICO': 'float64','CO_TURNO_ALUNO': 'float64',
                              'TP_GRAU_ACADEMICO': 'float64','CO_OCDE_AREA_DETALHADA': 'float64',
                               'CO_OCDE_AREA_ESPECIFICA': 'float64','CO_OCDE_AREA_GERAL': 'float64',
                               'TP_TURNO': 'float64','IN_ING_CONVENIO_PECG': 'float64' ,
                              "CO_ALUNO" :"object" , "ID_ALUNO" : "object",'ANO_INGRESSO': 'float64',
       'CO_ALUNO_SITUACAO': 'float64',
       'CO_COR_RACA_ALUNO': 'float64',
       'CO_NACIONALIDADE_ALUNO': 'float64',
       'CO_PAIS_ORIGEM_ALUNO': 'float64',
       'CO_SEMESTRE_REFERENCIA': 'float64',
       'CO_TIPO_ESCOLA_ENS_MEDIO': 'float64',
       'IN_ALUNO_DEF_TGD_SUPER': 'float64',
       'IN_APOIO_SOCIAL': 'float64',
       'IN_ATIVIDADE_EXTRACURRICULAR': 'float64',
       'IN_CONCLUINTE': 'float64',
       'IN_INGRESSO_TOTAL': 'float64',
       'IN_INGRESSO_VAGA_NOVA': 'float64',
       'IN_ING_AVALIACAO_SERIADA': 'float64',
       'IN_ING_DECISAO_JUDICIAL': 'float64',
       'IN_ING_ENEM': 'float64',
       'IN_ING_SELECAO_SIMPLIFICADA': 'float64',
       'IN_ING_SELECAO_VAGA_PROG_ESPEC': 'float64',
       'IN_ING_SELECAO_VAGA_REMANESC': 'float64',
       'IN_ING_TRANSF_EXOFFICIO': 'float64',
       'IN_ING_VESTIBULAR': 'float64',
       'IN_MATRICULA': 'float64',
       'IN_RESERVA_VAGAS': 'float64',
       'IN_SEXO_ALUNO': 'float64',
       'NU_ANO_ALUNO_NASC': 'float64',
       'NU_DIA_ALUNO_NASC': 'float64',
       'NU_IDADE_ALUNO': 'float64',
       'NU_MES_ALUNO_NASC': 'float64',
       'QT_CARGA_HORARIA_INTEG': 'float64',
       'QT_CARGA_HORARIA_TOTAL': 'float64',
       'IN_RESERVA_ETNICO' : 'object' }

col_sel1= ["CO_IES","CO_ALUNO","CO_CATEGORIA_ADMINISTRATIVA","CO_CURSO",
           "CO_NIVEL_ACADEMICO","CO_COR_RACA_ALUNO",
           "IN_SEXO_ALUNO","IN_RESERVA_ETNICO","IN_RESERVA_VAGAS",
           "IN_APOIO_SOCIAL"]

col_sel2= ["CO_IES","ID_ALUNO","TP_CATEGORIA_ADMINISTRATIVA","CO_CURSO",
           "TP_NIVEL_ACADEMICO","TP_COR_RACA",
           "TP_SEXO","IN_RESERVA_ETNICO","IN_RESERVA_VAGAS",
           "IN_APOIO_SOCIAL"]


col_sel4= ["CO_IES","CO_ALUNO","TP_CATEGORIA_ADMINISTRATIVA","CO_CURSO",
           "TP_NIVEL_ACADEMICO","TP_COR_RACA",
           "TP_SEXO","IN_RESERVA_ETNICO","IN_RESERVA_VAGAS",
           "IN_APOIO_SOCIAL"]


O Código que realiza isto está abaixo.

In [ ]:
datasets = []
count = 0

                      
anos = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
                
for arquivo in arquivos :
   pedaco =dd.read_csv(arquivo,sep = "|",encoding="ISO-8859-1",dtype = types)
                       

   if count in [0,1] :
    pedaco = pedaco[col_sel1]
   elif count in [2,3,4,5,6,7]  :
    pedaco = pedaco[col_sel1]

   elif count in [8] :
    pedaco = pedaco[col_sel4]
    

   else:
    pedaco = pedaco[col_sel2]


  
   pedaco["Ano"] = anos[count]
   colunas = ["CO_IES","CO_ALUNO","Cat_Adm","CO_CURSO","Nivel_Aca",
              "Cor/Raca","Genero","IN_RESERVA_ETNICO","IN_RESERVA_VAGAS","Apoio_Social","Ano"]
   pedaco.columns =colunas
   
  
   datasets.append(pedaco)
   count += 1


Vamos então unir os datasets, pois padronizamos as colunas. Para isto usaremos a função concat.

In [ ]:
dataAluno = dd.concat(datasets)


Temos cerca de 100 milhões de entradas, o que não é um fato correspondente com a realidade de alunos. Precisamos diminuir, eliminando o mesmo aluno, se contato duas vezes, vamos fazer isto, utilizando o método drop_duplicates do dask. Esse processo é longo, pois tem que gerar por todo o dataset.


In [ ]:
print(f'Dataset com Duplicatas possui {len(dataAluno)} linhas')
dataAluno = dataAluno.drop_duplicates("CO_ALUNO")
datafinal = dataAluno.compute()
print(f'Dataset sem duplicatas possui {(datafinal.shape[0])} linhas')



Dataset com Duplicatas possui 113190205 linhas
Dataset sem duplicatas possui 44261224 linhas


##Mexendo com Curso

Na tabela de cursos nos interessa a princípio, apenas o identificador do Curso e o nome dele(NO_CURSO). Neste caso não temos diferenças nas nomeclaturas, o que facilita o processo

In [ ]:
col_sel = ["CO_CURSO","NO_CURSO"] #criando as variaveis
types = {'CO_MUNICIPIO_CURSO': 'float64',
       'CO_UF_CURSO': 'float64',
       'IN_OFERECE_DISC_DISTANCIA': 'float64',
       'QT_INSCRITOS_ANO': 'float64',
       'CO_OCDE_AREA_DETALHADA': 'float64',
       'NO_MUNICIPIO_CURSO': 'object',
       'NO_REGIAO_CURSO': 'object',
       'QT_INSCRITOS_ANO_EAD': 'float64',
       'QT_VAGAS_ANUAL_EAD': 'float64',
       'SGL_UF_CURSO': 'object',
       'CO_OCDE_AREA_ESPECIFICA': 'float64',
       'CO_OCDE_AREA_GERAL': 'float64',
       'IN_GRATUITO': 'float64',
       'CO_GRAU_ACADEMICO': 'float64',
       'IN_INTEGRAL_CURSO': 'float64',
       'IN_MATUTINO_CURSO': 'float64',
       'IN_NOTURNO_CURSO': 'float64',
       'IN_VESPERTINO_CURSO': 'float64',
       'QT_CONCLUINTE_CURSO': 'float64',
       'TP_ATRIBUTO_INGRESSO': 'float64'
}

Criando o datasets por ano

In [ ]:
datasets = []
count = 0

                      
anos = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
                
for arquivo in arquivos_CURSO :
   pedaco =dd.read_csv(arquivo,sep = "|",encoding="ISO-8859-1",dtype = types)                
   pedaco = pedaco[col_sel]
                                    
   colunas = ["CO_CURSO","NO_CURSO"]
   pedaco.columns =colunas
   datasets.append(pedaco)
   count += 1



Concatenando numa variável

In [ ]:
dataCurso = dd.concat(datasets)
dataCurso.head()

,CO_CURSO,NO_CURSO
0,10394,ENFERMAGEM
1,5615,DIREITO
2,5618,LETRAS
3,18064,HISTORIA
4,5625,ENGENHARIA QUIMICA


Ao contrário dos Alunos, temos um dataset relativamente pequeno, não precisando, desta forma, mudar os  tipos. O trabalho com cursos está feito.

Vamos utilizar um método semelhante para os cursos e IES

##Mexendo com IES

Neste dataset eu tenho interesse em algumas variáveis a saber :

* CO_IES - Identificador do Instituto

* NO_IES - Nome do Instituto

* CO_MUNICIPIO_IES - Municipio onde se localiza o Instituto

* CO_UF_IES - Estado onde se localiza o Instituto

* IN_CAPITAL_IES - Indica se o Instituto esta localizado em uma capital do país.



In [ ]:
col_sel1 = ['CO_IES', 'NO_IES', 'CO_MUNICIPIO_IES', 'CO_UF_IES',
        'IN_CAPITAL_IES']
col_sel2 =  ['CO_IES', 'NO_IES', 'CO_MUNICIPIO', 'CO_UF',
        'IN_CAPITAL']

Gerando o dataset por ano

In [ ]:
datasets = []
count = 0

                      
anos = [2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019]
                
for arquivo in arquivos_IES :
   pedaco =dd.read_csv(arquivo,sep = "|",encoding="ISO-8859-1")   
   if count <8 :             
    pedaco = pedaco[col_sel1]
   else :
    pedaco =pedaco[col_sel2]
                                    
   colunas = ['CO_IES', 'NO_IES', 'CO_MUNICIPIO_IES', 'CO_UF_IES',
        'IN_CAPITAL_IES']
   pedaco.columns =colunas
   datasets.append(pedaco)
   count += 1


Juntando os datasets

In [ ]:
dataIES = dd.concat(datasets)
dataIES.head()

,CO_IES,NO_IES,CO_MUNICIPIO_IES,CO_UF_IES,IN_CAPITAL_IES
0,699,FUNDACAO UNIVERSIDADE FEDERAL DE RONDONIA,1100205,11,1
1,1028,UNIVERSIDADE ESTADUAL DE MATO GROSSO DO SUL,5003702,50,0
2,1189,UNIVERSIDADE DO PLANALTO CATARINENSE,4209300,42,0
3,275,UNIVERSIDADE DO VALE DO PARAIBA,3549904,35,0
4,580,UNIVERSIDADE FEDERAL DE PERNAMBUCO,2611606,26,1


In [ ]:
dataIES.info

<bound method DataFrame.info of Dask DataFrame Structure:
               CO_IES  NO_IES CO_MUNICIPIO_IES CO_UF_IES IN_CAPITAL_IES
npartitions=11                                                         
                int64  object            int64     int64          int64
                  ...     ...              ...       ...            ...
...               ...     ...              ...       ...            ...
                  ...     ...              ...       ...            ...
                  ...     ...              ...       ...            ...
Dask Name: concat, 66 tasks>

Olhando para os tipos de dados, temos que a variavel IN_CAPITAL_IES está como float64, sendo que ela aceita apenas 2 valores. Vamos transformá-las em uma coluna boolena.

In [ ]:
dataIES.IN_CAPITAL_IES= dataIES.IN_CAPITAL_IES.astype('bool')


Finalizando a criação dos 3 datasets vamos uní-los em apenas um com as informações úteis.

## Juntando os datasets e fazendo uma pré limpeza


Vamos olhar o início dos nossos datasets

In [ ]:
print(dataIES.head(1))
print(dataCurso.head(1))
print(datafinal.head(1))

   CO_IES                                     NO_IES  ...  CO_UF_IES  IN_CAPITAL_IES
0     699  FUNDACAO UNIVERSIDADE FEDERAL DE RONDONIA  ...         11            True

[1 rows x 5 columns]
   CO_CURSO    NO_CURSO
0     10394  ENFERMAGEM
   CO_IES      CO_ALUNO  Cat_Adm  ...  IN_RESERVA_VAGAS  Apoio_Social   Ano
0     780  400000004203        4  ...               0.0           0.0  2009

[1 rows x 11 columns]


Vemos que no dataset de Cursos e no dataAluno temos em ambos as variáveis CO_CURSO. Como Curso tem apenas duas colunas, podemos fazer um dicionário com estas colunas, fazer uma associação e colocar a coluna Curso no dataset Aluno. Vamos criar uma função para fazer isto pois pode ser úil futuramente. Importante notar que a função criada utiliza dataframe do pandas, logo precisamos transformar o dataframe do dask em um dataframe do pandas atraves do método compute.

In [ ]:
#@Criando a função
def dictColuna(datasetvelho,datasetNovo,coluna1,coluna2,nomeNovaColuna) :
  datax = datasetvelho.set_index(coluna1).to_dict()[coluna2]
  a = datasetNovo[coluna1].map(datax)
  datasetNovo[nomeNovaColuna] = a
  

In [ ]:
dataCursoLimpo = dataCurso.compute()

Aplicando a função

In [ ]:
dictColuna(dataCursoLimpo,datafinal,"CO_CURSO","NO_CURSO","Nome_Curso")



Vendo o resultado 

In [ ]:
datafinal.head()

,CO_IES,CO_ALUNO,Cat_Adm,CO_CURSO,Nivel_Aca,Cor/Raca,Genero,IN_RESERVA_ETNICO,IN_RESERVA_VAGAS,Apoio_Social,Ano,Nome_Curso
0,780,400000004203,4,18471,1,2.0,1.0,0,0.0,0.0,2009,ARQUITETURA E URBANISMO
1,570,400000004213,1,40171,1,6.0,0.0,0,0.0,0.0,2009,ENGENHARIA DA COMPUTAÇÃO
2,316,400000004239,4,82227,1,0.0,1.0,0,0.0,0.0,2009,ODONTOLOGIA
3,670,400000004262,6,15474,1,6.0,0.0,0,0.0,0.0,2009,ADMINISTRAÇÃO
4,1131,400000004269,4,86830,1,1.0,1.0,0,0.0,0.0,2009,ENFERMAGEM


Vemos que foi adicionada uma coluna com o Curso. O Código funcionou bem, faremos parecido com os dados de IES

No dataset referente as IES, temos as variáveis CO_MUNICIPIO_IES e CO_UF_IES que são os códigos do IBGE para cidades e estados. O código sozinho não nos dá uma informação útil, vamos utilizar a função criada anteriormente com uma tabela de correlação, para transformar estes Códigos em Cidades e Estados que dão informação

In [ ]:
url = "https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/estados.csv" #estados IBGE
datasEstados = pd.read_csv(url,sep=",",usecols = ["codigo_uf","uf"])
url =  "https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv" #municipios
datasMunicipios = pd.read_csv(url,sep = ",", usecols = ["codigo_ibge","nome"])
datasEstados.columns = ["CO_UF_IES","uf"]
datasMunicipios.columns = ["CO_MUNICIPIO_IES","cidade"]



Utilizando o código para os estados e municipios

In [ ]:
dictColuna(datasEstados,dataIES,"CO_UF_IES","uf","UF")
dictColuna(datasMunicipios,dataIES,"CO_MUNICIPIO_IES","cidade","Cidade")


In [ ]:
dataIES.head()

,CO_IES,NO_IES,CO_MUNICIPIO_IES,CO_UF_IES,IN_CAPITAL_IES,UF,Cidade
0,699,FUNDACAO UNIVERSIDADE FEDERAL DE RONDONIA,1100205,11,True,RO,Porto Velho
1,1028,UNIVERSIDADE ESTADUAL DE MATO GROSSO DO SUL,5003702,50,False,MS,Dourados
2,1189,UNIVERSIDADE DO PLANALTO CATARINENSE,4209300,42,False,SC,Lages
3,275,UNIVERSIDADE DO VALE DO PARAIBA,3549904,35,False,SP,São José dos Campos
4,580,UNIVERSIDADE FEDERAL DE PERNAMBUCO,2611606,26,True,PE,Recife


Com essas modificações, as colunas CO_UF_IES e CO_MUNICIPIO_IES não se tornam mais úteis, iremos dropa-las.

In [ ]:
dataIESClean = dataIES.drop(columns = ["CO_UF_IES","CO_MUNICIPIO_IES"])


Semelhante ao Curso, temos que CO_IES aparece em ambos os datasets, tanto referente aos Institutos, quanto referente aos Alunos. Vamos utilizar nosso código de dicionário para juntar as informações relevantes.


In [ ]:
dataIESClean = dataIESClean.compute()

In [ ]:
colunasVelhas = ["NO_IES","Cidade","UF","IN_CAPITAL_IES"]
colunasNovas = ["Nome_Instituição","Municipio","UF","CAPITAL"]
for number in range(len(colunasVelhas)) :
  dictColuna(dataIESClean,datafinal,"CO_IES",colunasVelhas[number],colunasNovas[number])



Vamos olhar o dataset Resultante da junção dos datasets.

In [ ]:
datafinal.head()


,CO_IES,CO_ALUNO,Cat_Adm,CO_CURSO,Nivel_Aca,Cor/Raca,Genero,IN_RESERVA_ETNICO,IN_RESERVA_VAGAS,Apoio_Social,Ano,Nome_Curso,Nome_Instituição,Municipio,UF,CAPITAL
0,780,400000004203,4,18471,1,2.0,1.0,0,0.0,0.0,2009,ARQUITETURA E URBANISMO,UNIVERSIDADE DE CUIABÁ,Cuiabá,MT,True
1,570,400000004213,1,40171,1,6.0,0.0,0,0.0,0.0,2009,ENGENHARIA DA COMPUTAÇÃO,UNIVERSIDADE FEDERAL DO RIO GRANDE DO NORTE,Natal,RN,True
2,316,400000004239,4,82227,1,0.0,1.0,0,0.0,0.0,2009,ODONTOLOGIA,UNIVERSIDADE NOVE DE JULHO,São Paulo,SP,True
3,670,400000004262,6,15474,1,6.0,0.0,0,0.0,0.0,2009,ADMINISTRAÇÃO,UNIVERSIDADE SÃO FRANCISCO,Bragança Paulista,SP,False
4,1131,400000004269,4,86830,1,1.0,1.0,0,0.0,0.0,2009,ENFERMAGEM,CENTRO UNIVERSITÁRIO SANTO AGOSTINHO,Teresina,PI,True


Por fim irei criar uma nova coluna, que indicar se o aluno ingressou na faculdade através de Cotas Raciais. Este dado não é igual ao dado referente em IN_Reserva_Etnico pois esta coluna possui vários valores nulos, que não foram prenchidos pois em IN_Reserva_Vagas já foi indicado uma negativa. Para um aluno se enquadrar na categoria de Cota Racial ele precisa prencher dois requisitos :

1  IN_Reserva_Vagas = True

2  IN_Reserva_Etnico = 1

Vamos fazer isto no código abaixo.

In [ ]:
print(f"Quantidade de Valores Nulos em IN_Reserva_Etnico \n {datafinal.IN_RESERVA_ETNICO.isna().sum()} ")
datafinal['Cota_Racial'] = 0 #iniciando a coluna
datafinal.loc[((datafinal["IN_RESERVA_ETNICO"] ==  '1') & (datafinal["IN_RESERVA_VAGAS"] == True)),
                     "Cota_Racial"] = 1            
datafinal.head(1)

Quantidade de Valores Nulos em IN_Reserva_Etnico 
 35791578 


,CO_IES,CO_ALUNO,Cat_Adm,CO_CURSO,Nivel_Aca,Cor/Raca,Genero,IN_RESERVA_ETNICO,IN_RESERVA_VAGAS,Apoio_Social,Ano,Nome_Curso,Nome_Instituição,Municipio,UF,CAPITAL,Cota_Racial
0,780,400000004203,4,18471,1,2.0,1.0,0,0.0,0.0,2009,ARQUITETURA E URBANISMO,UNIVERSIDADE DE CUIABÁ,Cuiabá,MT,True,0


O dataset parece funcionar como o esperado, com todas as informações que queremos para nossa análise, que será feita em outro notebook.


##Últimos Ajustes

Vamos ver os tipos de dados, e ver se podermos melhorar algum para ocupar menos espaço.


In [ ]:
datafinal.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44261224 entries, 0 to 90268
Data columns (total 17 columns):
 #   Column             Dtype  
---  ------             -----  
 0   CO_IES             int64  
 1   CO_ALUNO           object 
 2   Cat_Adm            int64  
 3   CO_CURSO           int64  
 4   Nivel_Aca          int64  
 5   Cor/Raca           float64
 6   Genero             float64
 7   IN_RESERVA_ETNICO  object 
 8   IN_RESERVA_VAGAS   float64
 9   Apoio_Social       float64
 10  Ano                int64  
 11  Nome_Curso         object 
 12  Nome_Instituição   object 
 13  Municipio          object 
 14  UF                 object 
 15  CAPITAL            bool   
 16  Cota_Racial        int64  
dtypes: bool(1), float64(4), int64(6), object(6)
memory usage: 5.6+ GB


Vemos que  o armazenamento de  variáveis está ocupando espaço desnecessário. Por exemplo Apoio_Social,IN_RESERVA_VAGAS, e Cota_Racial são variáveis booleana
e eles estão armazenadas como float64 ocupando muito espaço. Semelhantemente ele está usando int64 para
a variável Cat_Adm e Nivel_Academico. Podemos armazená-las em int8 sem problemas de armazenamento e pouparemos memória dessa forma. Por fim podemos retirar as variáveis referentes a códigos, pois temos os seus valores reais. Faremos isto na celula abaixo.


In [ ]:
datafinal.IN_RESERVA_VAGAS = datafinal.IN_RESERVA_VAGAS.astype("bool")
datafinal.Apoio_Social = datafinal.Apoio_Social.astype("bool")
datafinal.Cota_Racial = datafinal.Cota_Racial.astype("bool")
datafinal.Cat_Adm = datafinal.Cat_Adm.astype("int8")
datafinal.Nivel_Aca = datafinal.Nivel_Aca.astype("int8")
datafinal.Genero = datafinal.Genero.astype("int8")
datafinal["Cor/Raca"] = datafinal["Cor/Raca"].astype("int8")

data = datafinal.drop(columns = ["CO_CURSO","CO_IES","CO_ALUNO","IN_RESERVA_ETNICO"])




Vamos checar o uso de memória e ver se criamos algum valor nulo na nossa transformação

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44261224 entries, 0 to 90268
Data columns (total 13 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Cat_Adm           int8  
 1   Nivel_Aca         int8  
 2   Cor/Raca          int8  
 3   Genero            int8  
 4   IN_RESERVA_VAGAS  bool  
 5   Apoio_Social      bool  
 6   Ano               int64 
 7   Nome_Curso        object
 8   Nome_Instituição  object
 9   Municipio         object
 10  UF                object
 11  CAPITAL           bool  
 12  Cota_Racial       bool  
dtypes: bool(4), int64(1), int8(4), object(4)
memory usage: 2.3+ GB


In [ ]:
data.isna().sum()

Cat_Adm             0
Nivel_Aca           0
Cor/Raca            0
Genero              0
IN_RESERVA_VAGAS    0
Apoio_Social        0
Ano                 0
Nome_Curso          0
Nome_Instituição    0
Municipio           0
UF                  0
CAPITAL             0
Cota_Racial         0
dtype: int64

Sem valores nulos e conseguimos liber 3GB de espaço(de 5.3 para 2.3). Parece que os dados estão prontos para serem armazenados e analisados em outro notebook. Por fim vamos ver o tempo que gastou para o código ser compilado.

In [ ]:
tempoSem = datetime.now() - startTime
print(f"Tempo gasto antes da conversão do dataframe para um arquivo .csv : {tempoSem} ")


Tempo gasto antes da conversão do dataframe para um arquivo .csv : 1:46:18.186974 


#Armazenamento no google drive

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44261224 entries, 0 to 90268
Data columns (total 13 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Cat_Adm           int8  
 1   Nivel_Aca         int8  
 2   Cor/Raca          int8  
 3   Genero            int8  
 4   IN_RESERVA_VAGAS  bool  
 5   Apoio_Social      bool  
 6   Ano               int64 
 7   Nome_Curso        object
 8   Nome_Instituição  object
 9   Municipio         object
 10  UF                object
 11  CAPITAL           bool  
 12  Cota_Racial       bool  
dtypes: bool(4), int64(1), int8(4), object(4)
memory usage: 2.3+ GB


Vemos que este dataset ia ocupar pelo menos 2,3GB de memória. Esta quantidade pode ser armazenada na maioria dos agregadores de dados atuais. Eu utilizarei o Google Drive para isto.

In [ ]:
from google.colab import drive #importando o google drive para esta sessão
drive.mount('/content/drive')

Mounted at /content/drive


Criarei uma pasta no meu google drive chamado dados_INEP e armazanei os dados como dataCkeab.csv.

In [ ]:
data.to_csv("/content/drive/MyDrive/dados_INEP_LIMPO/dataClean.csv",chunksize=  100000  ,encoding="ISO-8859-1", index=False)


Por fim vamos limpar os dados criados por ocupam muito espaço e já temos os dados que desejávamos extrair.

In [ ]:
print('Fazendo a limpeza de dados ociosos')
for data in diretorio :
    shutil.rmtree(data)
for data in zip_files :
    os.remove(data)


print("Limpeza de dados ociosos completa!")
tempo = datetime.now()- startTime

print(f"A duração de execucação de todo o notebook durou :{tempo } ")


Fazendo a limpeza de dados ociosos
Limpeza de dados ociosos completa!
A duração de execucação de todo o notebook durou :2:24:53.406702 
